#0 Входные данные варианта лабораторной работы.

In [ ]:
# User categories
category_1 = 'Autouser'
category_2 = 'Businessuser'
category_3 = 'Homeuser'
category_4 = 'Bookuser'
# Analysed domains 
domains_category_1 = [u'cars.ru', u'avto-russia.ru', u'bmwclub.ru']
domains_category_2 = [u'zakon.kz', u'egov.kz', u'makler.md']
domains_category_3 = [u'russianfood.com', u'psychologies.ru', u'gotovim.ru']
domains_category_4 = [u'books.imhonet.ru', u'zhurnaly.biz', u'zvukobook.ru']

# User identificators by categories
id1 = 2
id2 = 3
id3 = 4
id4 = 5

#1 Обработка исходных файлов для получения данных.

In [ ]:
#!/opt/anaconda/envs/bd9/bin/python

#Command for launching python file
'''
hadoop jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapred.reduce.tasks=0 \
-input /labs/lab03data/* \
-output /user/valeria.lupanova/lab03/input \
-mapper "/opt/anaconda/envs/bd9/bin/python lab03.py" \
-file "./lab03.py"
'''

from urllib.parse import urlparse, unquote
import sys
import re

def url2domain(url):
    try:
        a = urlparse(unquote(url.strip()))
        if (a.scheme in ['http','https']):
            b = re.search("(?:www\.)?(.*)",a.netloc).group(1)
            if b is not None:
                return str(b).strip()
            else:
                return ''
        else:
            return ''
    except:
        return ''

def mapper(line):
    
    uid, ts, url = line.split('\t') 
    
    if (uid is not None and url is not None and
        (re.match(r'^http.+$', url) or re.match(r'^https.+$', url))):
        domain = url2domain(url)
        print('{}\t{}\t{}'.format(uid, ts, domain))

                    
def main():

    for line in sys.stdin:
        try:
            mapper(line)
        except ValueError:
            continue


if __name__ == '__main__':
    main()

#2 Создание таблицы для работы с обработанными исходными данными.

In [ ]:
create external table valeria_lupanova.raw_data
(uid string    comment 'Уникальный идентификатор пользователя', 
 ts string     comment 'Отметка времен в UNIX формате', 
 domain string comment 'Экранированный URL')
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03/input';

In [ ]:
msck repair table valeria_lupanova.raw_data;  
select * from valeria_lupanova.raw_data limit 100;

#3 Создание таблицы для результирующих данных.

In [ ]:
create external table valeria_lupanova.valeria_lupanova 
(uid            bigint, 
 user_cat1_flag bigint, 
 user_cat2_flag bigint,  
 user_cat3_flag bigint, 
 user_cat4_flag bigint)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03/output';

#4 Запись результирующих данных в output-директорию.

In [ ]:
INSERT OVERWRITE DIRECTORY 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03/output'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
select distinct a.uid as uid, 
case when a.domain in ('cars.ru', 'avto-russia.ru', 'bmwclub.ru') and a.cnt >= 10 then 1 else 0 end as user_cat1_flag, 
case when a.domain in ('zakon.kz', 'egov.kz', 'makler.md') and a.cnt >= 10 then 1 else 0 end as user_cat2_flag,
case when a.domain in ('russianfood.com', 'psychologies.ru', 'gotovim.ru') and a.cnt >= 10 then 1 else 0 end as user_cat3_flag,
case when a.domain in ('books.imhonet.ru', 'zhurnaly.biz', 'zvukobook.ru') and a.cnt >= 10 then 1 else 0 end as user_cat4_flag from 
(select uid, domain, count(*) as cnt from raw_data group by uid, domain) a
order by a.uid;

#5 Перенос данных в локальную директорию.

In [ ]:
hdfs dfs -cat /user/valeria.lupanova/lab03/output/* > ~/lab03_users.txt

# Review

1. Непонятно, почему был выбран ipynb формат
2. Не ясно для чего: msck repair table valeria_lupanova.raw_data;  
3. Команды указаны верные
4. Оформление и комментарии - nice